In [ ]:
!pip install gdown==v5.1.0
!pip install Pillow
!pip install split-folders
!pip install lightning

In [ ]:
import lightning

from importlib.machinery import SourceFileLoader
from google.colab import output
from pytorch_lightning.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from google.colab import output
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
! git clone https://github.com/xpaf/MusicNet

Cloning into 'MusicNet'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 60 (delta 13), reused 56 (delta 9), pack-reused 0
Receiving objects: 100% (60/60), 12.04 KiB | 6.02 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
DM = SourceFileLoader('dm', "/content/MusicNet/GtzanDataModule.py").load_module()
ResMod = SourceFileLoader('rm', "/content/MusicNet/models/ResModel.py").load_module()
MC = SourceFileLoader('mc', "/content/MusicNet/MusicGenreClassifier.py").load_module()

In [ ]:
gtzan_dm = DM.GtzanDataModule(12, 16)
model = ResMod.ResModel()
classifier = MC.MusicClassifier(model)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
LOG_DIR = "./logs"
MODEL_NAME = "ResModel"
logger = TensorBoardLogger(LOG_DIR, name=MODEL_NAME)

In [ ]:
MODEL_CALLBACK = ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'model_checkpoint-{MODEL_NAME}/',
    filename='model-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}'
)

In [ ]:
! nvidia-smi

In [ ]:
trainer = lightning.Trainer(max_epochs=30,
                     check_val_every_n_epoch=5,
                     logger=logger,
                     log_every_n_steps=2,
                     accelerator="gpu",
                     callbacks=[MODEL_CALLBACK]
)

trainer.fit(model=classifier, datamodule=gtzan_dm)

In [ ]:
trainer.validate(datamodule=gtzan_dm)

In [ ]:
trainer.test(datamodule=gtzan_dm)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/logs/ResModel

In [ ]:
output.serve_kernel_port_as_window(6006, path="")

In [ ]:
labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
y_true_list = []
y_pred_list = []
device = "cuda"
data_module.setup(stage="test")
model = model.to(device)
model.eval()
with torch.no_grad():
  for x, y in data_module.test_dataloader():
    x, y = x.to(device), y.to(device)
    pred = model(x).argmax(1)

    for s_y, s_pred in zip(y, pred):
      y_true_list.append(int(s_y))
      y_pred_list.append(int(s_pred))

In [ ]:
cm = confusion_matrix(y_true_list, y_pred_list)
cmp = ConfusionMatrixDisplay(cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(10,10))
ax.set_title('Predicted genres')
cmp.plot(ax=ax)

In [ ]:
cmn = cm.astype('float')*100 / cm.sum(axis=1)[:, np.newaxis]
cmpn = ConfusionMatrixDisplay(cmn, display_labels=labels)
fig, ax = plt.subplots(figsize=(10,10))
ax.set_title('Percent')
cmpn.plot(ax=ax)